Use GPT2 models

In [ ]:
import numpy as np
import pandas as pd

from datasets import Dataset

from transformers import GPT2Tokenizer, GPT2ForSequenceClassification

import duckdb

### Load iris data

In [ ]:
con = duckdb.connect("../imdb.db")
imdb = con.sql("SELECT * FROM imdb").df()
con.close()

imdb['label'] = imdb['label'].astype(int)
imdb.head()

In [ ]:
imdb_test = Dataset.from_pandas(imdb[imdb['stage'] == 'test'].drop(columns=['stage']))

### Train the model

In [ ]:
model_name = 'gpt2-imdb-finetune'
model_path = f"/homes/ukumaras/scratch/Models/{model_name}"
# model_path = f"hipnologo/gpt2-imdb-finetune"

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2ForSequenceClassification.from_pretrained(model_path)

In [ ]:
imdb_test['text']

In [ ]:
tokenizer.padding_side = "left"

# Define PAD Token = EOS Token = 50256
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# encoding the input text
input = tokenizer.batch_encode_plus(imdb_test['text'][:100], return_tensors="pt", padding=True, truncation=True, max_length=512)
input

# # Move the input_ids tensor to the same device as the model
# input_ids = input_ids.to(model.device)

# # getting the logits 
logits = model(**input).logits

In [ ]:
# getting the predicted class
predicted_class = logits.argmax(-1)

In [ ]:
predicted_class

In [ ]:
imdb_test['label']